In [8]:
import torch
from diffusers import DiffusionPipeline

torch.backends.cuda.matmul.allow_tf32 = True
torch._inductor.config.conv_1x1_as_mm = True
torch._inductor.config.coordinate_descent_tuning = True
torch._inductor.config.epilogue_fusion = False
torch._inductor.config.coordinate_descent_check_all_directions = True

In [9]:
prompt = "A high-quality photo of a confused bear in calculus class. The bear is wearing a party hat and steampunk armor."
seed = 42

In [ ]:
pipe_sd_turbo = DiffusionPipeline.from_pretrained(
    "stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16"
).to("cuda")

In [ ]:
pipe_sd.unet.down_blocks[2].attentions[1]

In [ ]:
image = pipe_sd_turbo(
    prompt,
    num_inference_steps=1,
    guidance_scale=0.0,
    generator=torch.Generator("cuda").manual_seed(seed),
).images[0]
image

In [ ]:
image.size

In [ ]:
pipe_sd = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
).to("cuda")

In [ ]:
with torch.no_grad():
    image_safe = pipe_sd(
        prompt,
        num_inference_steps=50,
        guidance_scale=7.5,
        generator=torch.Generator("cuda").manual_seed(seed),
        output_type="latent",
    ).images[0]


In [ ]:
image_safe.resize((512, 512))

In [11]:
from src.hooked_model.hooked_model_sdxl import HookedDiffusionModel

hooked_model = HookedDiffusionModel(
    model=pipe_sd_turbo.unet,
    scheduler=pipe_sd_turbo.scheduler,
    encode_prompt=pipe_sd_turbo.encode_prompt,
    vae=pipe_sd_turbo.vae,
)

In [12]:
with torch.no_grad():
    out = hooked_model(
        prompt,
        guidance_scale=0.0,
        num_inference_steps=1,
        height=512,
        width=512,
        generator=torch.Generator("cuda").manual_seed(seed),
    )

In [ ]:
out[0]